In [1]:
import zipfile

zip_file_path = "/content/drive/MyDrive/animals (2).zip"
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("/content/unzipped_folder")

# ✅ Updated path to 'raw-img' folder
dataset_path = "/content/unzipped_folder/animals/animals/raw-img"


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/animals (2).zip'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import numpy as np


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (128, 128)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    dataset_path, target_size=img_size, batch_size=batch_size,
    class_mode='categorical', subset='training', seed=42)

val_data = datagen.flow_from_directory(
    dataset_path, target_size=img_size, batch_size=batch_size,
    class_mode='categorical', subset='validation', seed=42)


Found 11665 images belonging to 6 classes.
Found 2913 images belonging to 6 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dropout(0.5),  # ✅ Dropout to reduce overfitting
    Dense(128, activation='relu'),
    Dense(train_data.num_classes, activation='softmax')
])


In [ ]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_11 (Conv2D)              │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,414 (12.61 MB)

 Trainable params: 3,305,414 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)


In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    callbacks=[early_stop]  # ⛔ Stops early if no improvement
)


Epoch 1/15
365/365 ━━━━━━━━━━━━━━━━━━━━ 29s 71ms/step - accuracy: 0.4034 - loss: 1.5163 - val_accuracy: 0.5881 - val_loss: 1.0568
Epoch 2/15
365/365 ━━━━━━━━━━━━━━━━━━━━ 23s 64ms/step - accuracy: 0.5952 - loss: 1.0590 - val_accuracy: 0.6571 - val_loss: 0.9297
Epoch 3/15
365/365 ━━━━━━━━━━━━━━━━━━━━ 42s 67ms/step - accuracy: 0.6628 - loss: 0.9026 - val_accuracy: 0.6711 - val_loss: 0.8666
Epoch 4/15
365/365 ━━━━━━━━━━━━━━━━━━━━ 25s 69ms/step - accuracy: 0.6972 - loss: 0.8017 - val_accuracy: 0.7075 - val_loss: 0.7992
Epoch 5/15
365/365 ━━━━━━━━━━━━━━━━━━━━ 25s 69ms/step - accuracy: 0.7387 - loss: 0.7201 - val_accuracy: 0.7206 - val_loss: 0.7669
Epoch 6/15
365/365 ━━━━━━━━━━━━━━━━━━━━ 39s 65ms/step - accuracy: 0.7586 - loss: 0.6562 - val_accuracy: 0.7357 - val_loss: 0.7369
Epoch 7/15
365/365 ━━━━━━━━━━━━━━━━━━━━ 25s 67ms/step - accuracy: 0.7874 - loss: 0.5725 - val_accuracy: 0.7274 - val_loss: 0.7600
Epoch 8/15
365/365 ━━━━━━━━━━━━━━━━━━━━ 23s 64ms/step - accuracy: 0.8123 - loss: 0.5123 - 

In [ ]:
import cv2
import numpy as np

def predict(model, image_path, class_indices):
    idx_to_class = {v: k for k, v in class_indices.items()}

    # Read and preprocess the image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))  # Resize to match the model's input size
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
    img = img.astype('float32') / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    # Predict the class
    pred = model.predict(img)[0]
    i = np.argmax(pred)  # Get the class with highest probability
    print(f"➡ {idx_to_class[i].upper()} ({pred[i]*100:.2f}%)")


In [ ]:
# Example usage:
image_path = "/content/piccat.jpg"  # Provide your image path here
predict(model, image_path, train_data.class_indices)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
➡ CAT (73.35%)


In [ ]:
# Example usage:
image_path = "/content/mako.jpg"  # Provide your image path here
predict(model, image_path, train_data.class_indices)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
➡ COW (47.22%)


In [ ]:
cxz
